<a href="https://colab.research.google.com/github/yuuiwqy622/unsupervised-segmentation/blob/main/unsupervised_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!pip install keras==2.1.5


TensorFlow 1.x selected.


In [ ]:
!rm -rf sample_data

In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 50.91 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [ ]:
!unzip reduced-isic.zip
!rm reduced-isic.zip

Archive:  reduced-isic.zip
   creating: reduced-isic/
   creating: reduced-isic/train_edge/
  inflating: reduced-isic/train_edge/ISIC_0000000.png  
  inflating: reduced-isic/train_edge/ISIC_0000001.png  
  inflating: reduced-isic/train_edge/ISIC_0000003.png  
  inflating: reduced-isic/train_edge/ISIC_0000006.png  
  inflating: reduced-isic/train_edge/ISIC_0000007.png  
  inflating: reduced-isic/train_edge/ISIC_0000009.png  
  inflating: reduced-isic/train_edge/ISIC_0000011.png  
  inflating: reduced-isic/train_edge/ISIC_0000013.png  
  inflating: reduced-isic/train_edge/ISIC_0000014.png  
  inflating: reduced-isic/train_edge/ISIC_0000015.png  
  inflating: reduced-isic/train_edge/ISIC_0000016.png  
  inflating: reduced-isic/train_edge/ISIC_0000018.png  
  inflating: reduced-isic/train_edge/ISIC_0000019.png  
  inflating: reduced-isic/train_edge/ISIC_0000020.png  
  inflating: reduced-isic/train_edge/ISIC_0000022.png  
  inflating: reduced-isic/train_edge/ISIC_0000023.png  
  inflating:

In [3]:
import pathlib
import os
import sys
import time
import h5py
import random

import numpy as np
import tensorflow as tf
import keras
from keras import losses
from keras import models
import matplotlib.pyplot as plt

from skimage import color, exposure, io, img_as_float, transform, filters, morphology, measure
from sklearn import metrics

Using TensorFlow backend.


In [ ]:
def load_images(filepath):    
    '''
    Loads the available images.
    '''
    data_root = pathlib.Path(os.path.join(filepath))
    image_paths = [str(path) for path in  list(data_root.glob('*'))]
    return image_paths

In [ ]:
def preprocess_images(image_paths, savedir, load_prev=False):
    '''
    Preprocessing for ISIC images
    '''
    if not load_prev or not os.path.isdir(savedir):
        os.makedirs(savedir, exist_ok=True)
        for path in image_paths:
            savepath = os.path.join(savedir, os.path.basename(path))
            print(path)
            im = img_as_float(io.imread(path))        
            im = transform.resize(im, (256, 256), mode="reflect", anti_aliasing=True)
            im = np.uint8(im * 255)
            io.imsave(fname=savepath,arr=im)

In [ ]:
preprocess_images(load_images('reduced-isic/train_img'), 'train_img', load_prev=False)

reduced-isic/train_img/ISIC_0000021.jpg
reduced-isic/train_img/ISIC_0000026.jpg
reduced-isic/train_img/ISIC_0000027.jpg
reduced-isic/train_img/ISIC_0000023.jpg
reduced-isic/train_img/ISIC_0000022.jpg
reduced-isic/train_img/ISIC_0000029.jpg
reduced-isic/train_img/ISIC_0000013.jpg
reduced-isic/train_img/ISIC_0000003.jpg
reduced-isic/train_img/ISIC_0000024.jpg
reduced-isic/train_img/ISIC_0000015.jpg
reduced-isic/train_img/ISIC_0000001.jpg
reduced-isic/train_img/ISIC_0000032.jpg
reduced-isic/train_img/ISIC_0000004.jpg
reduced-isic/train_img/ISIC_0000028.jpg
reduced-isic/train_img/ISIC_0000031.jpg
reduced-isic/train_img/ISIC_0000000.jpg
reduced-isic/train_img/ISIC_0000030.jpg
reduced-isic/train_img/ISIC_0000007.jpg
reduced-isic/train_img/ISIC_0000025.jpg
reduced-isic/train_img/ISIC_0000016.jpg
reduced-isic/train_img/ISIC_0000017.jpg
reduced-isic/train_img/ISIC_0000011.jpg
reduced-isic/train_img/ISIC_0000012.jpg
reduced-isic/train_img/ISIC_0000006.jpg
reduced-isic/train_img/ISIC_0000020.jpg


In [ ]:
!pip install dominate

In [ ]:
!cp -r reduced-isic/train_edge train_label

In [ ]:
# # Step 4: Train pix2pixHD and choose the best epoch using the Frechet Inception Distance
# # This uses pytorch instead of tensorflow, so we run these steps externally using the pix2pixHD code provided by nVidia
print('Training pix2pixhD with only training images... ')
start = time.time()
# # Train command: python train.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --dataroot pix2ultra/datasets/isic2018/ --nThreads 4 --display_winsize 256 --tf_log --no_instance --label_nc 2
!python pix2pixHD/train.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --dataroot ./ --nThreads 2 --display_winsize 256 --tf_log --no_instance --label_nc 2
# # FID command: python fid.py path/to/images path/to/other/images --gpu 0
end = time.time()
print('took {} seconds'.format(end-start))

Training pix2pixhD with only training images... 
------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: pix2ultra/checkpoints
continue_train: False
data_type: 32
dataroot: ./
debug: False
display_freq: 100
display_winsize: 256
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 2
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: pix2ultra
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: none
save_epoch_freq: 10
save_latest_freq: 1000


In [ ]:
!ls pix2ultra/checkpoints/pix2ultra
# !zip -r pix2pix-check pix2ultra/checkpoints/

200_net_D.pth  iter.txt		 latest_net_G.pth  loss_log.txt  web
200_net_G.pth  latest_net_D.pth  logs		   opt.txt


In [ ]:
# Step 5: Use the trained pix2pixHD model to generate synthetic ultrasound images from the synthetic edge diagrams
print(' all images... ')
start = time.time()
# Test command: python test.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --results_dir pix2ultra/results --how_many 3000 --dataroot pix2ultra/datasets/isic2018/ --display_winsize 256 --no_instance --label_nc 2
!python pix2pixHD/test.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --results_dir pix2ultra/results --how_many 30 --dataroot ./ --display_winsize 256 --no_instance --label_nc 2
end = time.time() 
print('took {} seconds'.format(end-start))

 all images... 
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: pix2ultra/checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./
display_winsize: 256
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 30
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 2
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: pix2ultra
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: pix2ultra/results
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset [Aligne

In [ ]:
!zip -r result.zip pix2ultra/results/pix2ultra/test_latest

  adding: pix2ultra/results/pix2ultra/test_latest/ (stored 0%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/ (stored 0%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/18_synthesized_image.jpg (deflated 2%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/3_input_label.jpg (deflated 58%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/12_synthesized_image.jpg (deflated 2%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/21_input_label.jpg (deflated 54%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/23_synthesized_image.jpg (deflated 2%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/14_input_label.jpg (deflated 58%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/21_synthesized_image.jpg (deflated 3%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/0_synthesized_image.jpg (deflated 2%)
  adding: pix2ultra/results/pix2ultra/test_latest/images/1_synthesized_image.jpg (deflated 2%)
  adding: pix2ultra/r

In [ ]:
!mv test_latest result

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 39.07 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [ ]:
!mv Mask_RCNN/mrcnn .
!rm -rf Mask_RCNN

In [ ]:
!unzip result.zip
!rm result.zip
!mv result/pix2ultra/results/result/test_latest res
!rm -rf result
!mv res result

Archive:  result.zip
   creating: result/
   creating: result/pix2ultra/
   creating: result/pix2ultra/results/
   creating: result/pix2ultra/results/result/
   creating: result/pix2ultra/results/result/test_latest/
   creating: result/pix2ultra/results/result/test_latest/images/
  inflating: result/pix2ultra/results/result/test_latest/images/18_synthesized_image.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/3_input_label.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/12_synthesized_image.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/21_input_label.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/23_synthesized_image.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/14_input_label.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/21_synthesized_image.jpg  
  inflating: result/pix2ultra/results/result/test_latest/images/0_synthesized_image.jpg  
  inflating

In [2]:
import os
import pathlib
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from imgaug import augmenters as iaa
from PIL import Image

from sklearn.model_selection import train_test_split

sys.path.append('ISIC2018/Mask_RCNN')
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log


class ShapesConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides values specific
    to the dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 1 images per GPU. We can put multiple images on each
    # GPU. Batch size is (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 3

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + lesion

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 64, 128, 256)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # set number of epoch
    STEPS_PER_EPOCH = 500

    # set validation steps 
    VALIDATION_STEPS = 50
    

class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    

class ISIC_Dataset(utils.Dataset):
    
    def load_shapes(self, mode, train_paths=None, val_paths=None):
        
        # Add classes
        self.add_class("shapes", 1, "lesion")
        
        if mode == "train":  
            for im in train_paths:
                self.add_image('shapes', image_id=os.path.basename(im), path=im)             
              
        if mode == "val":
            for im in val_paths:
                self.add_image('shapes', image_id=os.path.basename(im), path=im)

    def image_reference(self, image_id):
        '''
        Return the shapes data of the image.
        '''
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id, mask_dir):
        '''
        Generate instance masks for shapes of the given image ID.
        '''
        info = self.image_info[image_id]        
        path = os.path.join(mask_dir, f'{image_id}.png')
        # path = os.path.join(mask_dir, os.path.splitext(info.get("id"))[0] + "_segmentation.png")
        mask = imread(path, pilmode='L')
        mask = np.expand_dims(mask, axis=-1)
        
        # Map class names to class IDs.
        class_ids = np.ones((1,), dtype=np.int32)
        
        return mask, class_ids


class MRCNN():
    def __init__(self, train_dir, mask_dir, savedir='Models/MaskRCNN', 
                 model_name='mrcnn_new',
                 init_weights='coco',
                 im_size=(256,256,3),
                 batch_size=8,
                 epochs=100):
        '''
        Initialize the MRCNN object parameters.
        '''
        self.train_dir = train_dir
        self.mask_dir = mask_dir
#        self.val_dir = val_dir
        self.savedir = savedir
        self.im_size = im_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.model_name = model_name + '.h5'
        self.init_weights = init_weights
        
        if not os.path.exists(savedir):
            os.makedirs(savedir)

    def setup_data_paths(self):
        '''
        Obtains image and mask paths from directories.
        '''
        train_dir = self.train_dir
        mask_dir = self.mask_dir
#        val_dir = self.val_dir
        
        train_images = [str(path) for path in list(pathlib.Path(train_dir).glob('*.jpg'))]
        train_masks = [str(path) for path in list(pathlib.Path(mask_dir).glob('*.png'))]
#        val_images = [str(path) for path in list(pathlib.Path(val_dir).glob('*.png'))]
        
        train_images, train_masks, val_images, val_masks =\
            train_test_split(train_images, train_masks, test_size=0.2, random_state=42)
        
        print('Number of training examples: {}'.format(len(train_images)))
        print('Number of validation examples: {}'.format(len(val_images)))
        
        self.train_images = train_images
        self.train_masks = train_masks
        self.val_images = val_images
        self.val_masks = val_masks
        self.n_train = len(train_images)
        self.n_val = len(val_images)
        
    def _process_pathnames(self, fname, label_path):
        '''
        Load input image.
        '''
        img_str = tf.read_file(fname)
        img = tf.image.decode_png(img_str, channels=3)
        
        label_img_str = tf.read_file(label_path)
        label_img = tf.image.decode_png(label_img_str, channels=1)
        return img, label_img
    
    def load_model(self):
        '''
        Loads pretrained COCO weights.
        '''
        savedir = self.savedir
        init = self.init_weights
        
        # Create model in training mode
        config = ShapesConfig()
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=savedir)
            
        # Choose weights for init
        if init == 'coco':
            coco_path = os.path.join(savedir, 'mask_rcnn_coco.h5')    
            if not os.path.exists(coco_path):
                utils.download_trained_weights(coco_path)
                   
            model.load_weights(coco_path, by_name=True,
                               exclude=['mrcnn_class_logits', 
                                        'mrcnn_bbox_fc',
                                        'mrcnn_bbox', 
                                        'mrcnn_mask'])
    
        elif init == 'imagenet':
            model.load_weights(model.get_imagenet_weights(), by_name=True)
        
        elif init == 'last':
            model.load_weights(model.find_last(), by_name=True)
        
        self.model = model   
        self.config = config
        return model
    
    def prepare_dataset(self):
        '''
        Prepares a dataset for training or validation.
        '''
        self.setup_data_paths()
        
        train_images = self.train_images
        val_images = self.val_images
        
        train = ISIC_Dataset()
        train.load_shapes('train', train_paths=train_images)
        train.prepare()
        
        val = ISIC_Dataset()
        val.load_shapes('val', train_paths=None, val_paths=val_images)
        val.prepare()
        
        return train, val
        
    def train(self):
        '''
        Train the loaded model
        '''
        config = self.config
        epochs = self.epochs
        
        # get data and model
        traindata, valdata = self.prepare_dataset()
        model = self.load_model()
        
        # hardcoded augmentation approach for now
        augmentation = iaa.SomeOf((0, 2), [
                    iaa.Fliplr(0.5),
                    iaa.Flipud(0.5),
                    iaa.OneOf([iaa.Affine(rotate=90),
                               iaa.Affine(rotate=180),
                               iaa.Affine(rotate=270)]),
                    iaa.Multiply((0.8, 1.5)),
                    iaa.GaussianBlur(sigma=(0.0, 5.0))
                    ])
        
        # train model
        model.train(traindata, valdata,
                    learning_rate=config.LEARNING_RATE,
                    epochs=epochs,
                    augmentation=augmentation,
                    layers='all')
        
        self.model = model
        self.augmentation = augmentation
        
    def inference(self):
        '''
        Runs model in inference mode.
        '''
        config = InferenceConfig()
        model = self.model
        savedir = self.savedir
        
        model = modellib.MaskRCNN(mode='inference',
                                  config=config,
                                  model_dir=savedir)
        
        self.model = model
        self.inference_config = config

Using TensorFlow backend.


In [3]:
def use_mask_rcnn(train_path, mask_path, savedir):
    '''
    Trains the mask_rcnn model on given images.
    '''
    # Train MRCNN
    mrcnn = MRCNN(train_path, mask_path, savedir='Models/MaskRCNN', epochs=5)
    mrcnn.load_model()
    mrcnn.train()
        
    return preds, scores

In [ ]:
!unzip synth_.zip
!rm synth_.zip

Archive:  synth_.zip
   creating: synth_image/
  inflating: synth_image/0_synthesized_image.jpg  
  inflating: synth_image/1_synthesized_image.jpg  
  inflating: synth_image/2_synthesized_image.jpg  
  inflating: synth_image/3_synthesized_image.jpg  
  inflating: synth_image/4_synthesized_image.jpg  
  inflating: synth_image/5_synthesized_image.jpg  
  inflating: synth_image/6_synthesized_image.jpg  
  inflating: synth_image/7_synthesized_image.jpg  
  inflating: synth_image/8_synthesized_image.jpg  
  inflating: synth_image/9_synthesized_image.jpg  
  inflating: synth_image/10_synthesized_image.jpg  
  inflating: synth_image/11_synthesized_image.jpg  
  inflating: synth_image/12_synthesized_image.jpg  
  inflating: synth_image/13_synthesized_image.jpg  
  inflating: synth_image/14_synthesized_image.jpg  
  inflating: synth_image/15_synthesized_image.jpg  
  inflating: synth_image/16_synthesized_image.jpg  
  inflating: synth_image/17_synthesized_image.jpg  
  inflating: synth_image/18

In [ ]:
%tensorflow_version 1.x
!pip install keras==2.1.5

TensorFlow is already loaded. Please restart the runtime to change versions.
  Using cached https://files.pythonhosted.org/packages/ba/65/e4aff762b8696ec0626a6654b1e73b396fcc8b7cc6b98d78a1bc53b85b48/Keras-2.1.5-py2.py3-none-any.whl
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [4]:
# Step 6: Use Mask-RCNN to train on the synthetic images - test is done on competiion submission page
import time 
import tensorflow as tf

print(' all images... ')
start = time.time()
train_path = 'synth_image'
mask_path = 'synth_mask'
unet_preds, unet_metrics = use_mask_rcnn(train_path, mask_path, savedir='mrcnn_masks')
end = time.time()
print('took {} seconds'.format(end-start))

 all images... 





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead






Number of training examples: 24
Number of validation examples: 24

Starting at epoch 0. LR=0.001

Checkpoint Path: Models/MaskRCNN/shapes20210319T1718/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (Ba

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/5
499/500 [============================>.] - ETA: 0s - loss: 1.2559 - rpn_class_loss: 0.0045 - rpn_bbox_loss: 0.3151 - mrcnn_class_loss: 0.1462 - mrcnn_bbox_loss: 0.3411 - mrcnn_mask_loss: 0.4490

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


500/500 [==============================] - 236s 473ms/step - loss: 1.2545 - rpn_class_loss: 0.0045 - rpn_bbox_loss: 0.3146 - mrcnn_class_loss: 0.1462 - mrcnn_bbox_loss: 0.3406 - mrcnn_mask_loss: 0.4486 - val_loss: 2.8829 - val_rpn_class_loss: 0.0143 - val_rpn_bbox_loss: 1.3347 - val_mrcnn_class_loss: 0.3233 - val_mrcnn_bbox_loss: 0.6187 - val_mrcnn_mask_loss: 0.5919

Epoch 2/5
500/500 [==============================] - 178s 356ms/step - loss: 0.6390 - rpn_class_loss: 0.0014 - rpn_bbox_loss: 0.1068 - mrcnn_class_loss: 0.0933 - mrcnn_bbox_loss: 0.1431 - mrcnn_mask_loss: 0.2944 - val_loss: 4.0260 - val_rpn_class_loss: 0.0304 - val_rpn_bbox_loss: 1.5394 - val_mrcnn_class_loss: 0.6725 - val_mrcnn_bbox_loss: 0.8570 - val_mrcnn_mask_loss: 0.9267
Epoch 3/5
500/500 [==============================] - 178s 356ms/step - loss: 0.4098 - rpn_class_loss: 8.7752e-04 - rpn_bbox_loss: 0.0547 - mrcnn_class_loss: 0.0459 - mrcnn_bbox_loss: 0.0827 - mrcnn_mask_loss: 0.2255 - val_loss: 5.0637 - val_rpn_class_

NameError: ignored